# HyDE (Hypothetical Document Embeddings) for RAG

이 노트북은 Hypothetical Document Embeddings (HyDE)를 사용하여 Retrieval-Augmented Generation (RAG) 시스템의 검색 성능을 향상시키는 방법을 보여줍니다. HyDE는 주어진 질문에 대한 가상의 문서를 생성하고, 이 가상 문서의 임베딩을 사용하여 실제 문서를 검색하는 방식입니다.

## 1. 환경 설정 및 라이브러리 임포트

In [3]:
from dotenv import load_dotenv
import os

# .env 파일에서 환경 변수 로드
load_dotenv()

from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_chroma import Chroma
from langchain.chains import HypotheticalDocumentEmbedder, LLMChain
from langchain.prompts import PromptTemplate, ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

## 2. 모델 및 임베딩 초기화

In [4]:
# 임베딩 모델 초기화 (OpenAI)
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

# LLM 초기화 (Google Gemini Flash)
llm = ChatGoogleGenerativeAI(
    model = "gemini-2.0-flash",
)

## 3. ChromaDB 및 Retriever 설정

In [5]:
# ChromaDB 영속 디렉토리 설정
PERSIST_DIRECTORY = r"C:\Users\Sese\AI_Study_Record\RAG_AGENT\rag_0705\chroma_db"
COLLECTION_NAME = "html_docs"

# ChromaDB 로드 및 Retriever 생성
db = Chroma(
    persist_directory=PERSIST_DIRECTORY,
    embedding_function=embeddings,
    collection_name=COLLECTION_NAME,
)
retriever = db.as_retriever()

## 4. HyDE (Hypothetical Document Embedder) 체인 구성

HyDE는 질문에 대한 가상의 답변(문서)을 생성하고, 이 가상 문서의 임베딩을 사용하여 실제 문서를 검색합니다. 이를 통해 질문과 직접적으로 일치하지 않는 문서도 검색할 수 있습니다.

In [6]:
# HyDE 문서 생성을 위한 프롬프트 템플릿
template_hyde = """Please write a scientific paper passage to answer the question
Question: {question}
Passage:"""
prompt_hyde = ChatPromptTemplate.from_template(template_hyde)

# HyDE 체인: 질문 -> LLM (가상 문서 생성) -> 임베딩 -> 검색
generate_docs_for_retrieval = (
    prompt_hyde | llm | StrOutputParser() 
)

## 5. RAG (Retrieval-Augmented Generation) 체인 구성

HyDE를 통해 검색된 문서를 기반으로 최종 답변을 생성하는 RAG 체인입니다.

In [7]:
# RAG를 위한 프롬프트 템플릿
template_rag = """Answer the following question based on this context:

{context}

Question: {question}
"""
prompt_rag = ChatPromptTemplate.from_template(template_rag)

# 최종 RAG 체인: 검색된 문서 + 질문 -> LLM (답변 생성)
final_rag_chain = (
    prompt_rag
    | llm
    | StrOutputParser()
)

## 6. HyDE RAG 실행 및 결과 확인

In [10]:
# 테스트 질문
question = "뎅기열과 비슷한 매개체로 전파되는 질환은?"

print(f"질문: {question}\n")

# 1. HyDE를 사용하여 가상 문서 생성
print("\033[94m--- HyDE를 사용하여 가상 문서 생성 중 ---\033[0m")
generated_hyde_doc = generate_docs_for_retrieval.invoke({"question":question})
print(f"\033[94m------ 생성된 가상 문서 (HyDE) ---\033[0m\n{generated_hyde_doc}\n")

# 2. 생성된 가상 문서를 사용하여 실제 문서 검색
print("\033[94m--- 가상 문서를 사용하여 실제 문서 검색 중 ---\033[0m")
retrieved_docs = retriever.invoke(generated_hyde_doc)

print("""\n\033[94m--- 검색된 문서 ---\033[0m\n""")
for i, doc in enumerate(retrieved_docs):
    print(f"문서 {i+1}:")
    print(f"ID: {doc.metadata.get('id')}")
    print(f"Source: {doc.metadata.get('source')}")
    print(f"Content (첫 200자): {doc.page_content[:200]}...\n")

# 2. 검색된 문서를 기반으로 최종 답변 생성
print("\033[94m--- 최종 답변 생성 중 ---\033[0m")
result = final_rag_chain.invoke({"context":retrieved_docs,"question":question})

print("""\n
--- 최종 답변 --- 
""")
print(result)

질문: 뎅기열과 비슷한 매개체로 전파되는 질환은?

--- HyDE를 사용하여 가상 문서 생성 중 ---
------ 생성된 가상 문서 (HyDE) ---
Here's a scientific paper passage answering the question "뎅기열과 비슷한 매개체로 전파되는 질환은? (What diseases are transmitted by similar vectors to Dengue fever?)":

"Dengue fever (DF) is a mosquito-borne viral disease caused primarily by four serotypes of the dengue virus (DENV 1-4). These viruses are primarily transmitted to humans through the bites of infected *Aedes aegypti* and, to a lesser extent, *Aedes albopictus* mosquitoes. Due to the shared vector ecology of these *Aedes* species, several other arboviral diseases are frequently found in the same geographic regions as DF and share similar transmission patterns. **Chikungunya, Zika virus disease, and yellow fever are prominent examples of diseases transmitted by the same *Aedes aegypti* mosquito vector as dengue fever.**  These diseases often present with overlapping clinical symptoms, making accurate diagnosis challenging. Furthermore, co-infection with